In [1]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7

Found existing installation: unsloth 2026.2.1
Uninstalling unsloth-2026.2.1:
  Successfully uninstalled unsloth-2026.2.1
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-ycf89css
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-ycf89css
  Resolved https://github.com/unslothai/unsloth.git to commit 56c8f9662b1bc1fb50bcbe6bcc45ddffb0cdeb60
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2026.2.1-py3-none-any.whl size=441477 sha256=858ebdc745f20be5e769eed24b35c3ba83191585dc35a02c081fb832e0ecafd1
  Stored in directory: /tmp/pip-ephem-wheel-cache-2y36789a/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth


In [1]:
import os

# Force Python to only see the first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datasets import load_dataset, ClassLabel, Value
import yaml


def load_prompts(yaml_path="prompts.yaml"):
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

def transform_dataset_task(dataset, task, technique, tokenizer, yaml_path="/configs/prompts.yaml"):
    """
    Transforms the dataset by applying the specific prompt technique from the YAML.

    Args:
        dataset (DatasetDict or Dataset): The Hugging Face dataset.
        task (str): 'task_1_clarity' or 'task_2_evasion'.
        technique (str): 'default', 'persona', 'chain_of_thought', 'definition_aware'.
        yaml_path (str): Path to prompts.yaml.

    Returns:
        Dataset: The dataset with a new 'text' column containing the formatted prompt.
    """
    prompts = load_prompts(yaml_path)

    # Validation
    if task not in prompts:
        raise ValueError(f"Task '{task}' not found in yaml. Available: {list(prompts.keys())}")
    if technique not in prompts[task]:
        raise ValueError(f"Technique '{technique}' not found for {task}. Available: {list(prompts[task].keys())}")

    config = prompts[task][technique]
    system_prompt = config['system']

    if task == 'task_1_clarity':
        label_column = "clarity_label"
        unique_labels = sorted(set(dataset["train"][label_column]))
    elif task == 'task_2_evasion':
        label_column = "evasion_label"
        unique_labels = sorted(set(dataset["train"][label_column]))
    # Step 1: Temporarily cast to ClassLabel for stratification

    dataset = dataset['train'].cast_column(
        label_column,
        ClassLabel(names=unique_labels)
    )
    # Step 2: Stratified split
    dataset = dataset.train_test_split(
        test_size=0.1,
        stratify_by_column=label_column,
        seed=42,
    )
    dataset = dataset['test']

    # Step 3: Convert labels BACK to strings
    def convert_label_back(example):
        idx = int(example[label_column])   # convert "4" → 4
        example[label_column] = unique_labels[idx]
        return example

    dataset = dataset.cast_column(label_column, Value("string"))
    dataset = dataset.map(convert_label_back)

    def format_row(row):
        # 1. Extract inputs
        q = row['interview_question']
        a = row['interview_answer']
        p = row['president']
        d = row['date']
        raw_label = row[label_column]


        # 2. Format the full string
        # Note: We strip whitespace to prevent tokenization issues at boundaries
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"President: {p}\nDate: {d}\n{q}\n{a}"}, #Q, A are already in the dataset before each Question and Answer respectively
            {"role": "assistant", "content": str(raw_label)}
            ]

        # 3. Apply Template
        # tokenize=False gives you the formatted string back so you can debug it
        # add_generation_prompt=False because we include the assistant response (training)
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

        return {"text": formatted_text}

    # Apply transformation
    # remove_columns is optional but recommended if you only need the 'text' for training
    return dataset.map(format_row, remove_columns=dataset.column_names)

In [3]:
from unsloth import FastLanguageModel
import torch

def get_model_and_tokenizer(model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit", max_seq_length=2048,
                            dtype=None, load_in_4bit = True, output_dir='.'):
    """
    Loads the 4-bit Quantized Mistral model with Unsloth optimizations.
    Returns: (model, tokenizer) tuple
    """
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        load_in_4bit = load_in_4bit,    # Force 4-bit loading
        dtype=dtype # Explicitly pass torch_dtype to address KeyError
    )

    # Attach LoRA Adapters (Configuration)
    model = FastLanguageModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,       # Optimized for Unsloth
        bias = "none",
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
    )
    model.save_pretrained(output_dir)
    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from unsloth.chat_templates import get_chat_template

model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Forces Mistral v0.3 format
    )

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


def train_model(model, tokenizer, dataset, output_dir='lora.pth',
                training_args = TrainingArguments(
                    per_device_train_batch_size = 1,
                    gradient_accumulation_steps = 8,
                    warmup_steps = 5,
                    num_train_epochs = 1, # Set this for 1 full training run.
                    learning_rate = 2e-4,
                    fp16 = not is_bfloat16_supported(),
                    bf16 = is_bfloat16_supported(),
                    logging_steps = 1,
                    optim = "adamw_8bit",
                    weight_decay = 0.01,
                    lr_scheduler_type = "linear",
                    seed = 21,
                    output_dir = "outputs",
                    report_to = "none", # Use this for WandB etc
                    )
                ):
    """
    Executes the training loop.
    Args:
        model: Pre-loaded Unsloth model object
        tokenizer: Pre-loaded Unsloth tokenizer
        output_dir: Where to save the adapter
        subset_size: Limit for pilot testing
    """

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        dataset_text_field = "text",
        max_seq_length = 2048,
        dataset_num_proc = 2,
        packing = False,
        args = training_args,
    )
    torch.cuda.empty_cache()

    trainer.train()
    print(f"Saving adapter to {output_dir}")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [7]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_1_clarity', 'default', tokenizer, "/content/prompts.yaml")

In [8]:
train_model(model, tokenizer, dataset=subset, output_dir= "task1_default")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.627100
2,2.390900
3,2.562500
4,2.178200
5,2.288500
6,2.089400
7,2.043200
8,1.938200
9,1.986700
10,2.029800


Saving adapter to task1_default


In [9]:
!zip -r /content/task1_default.zip /content/task1_default/

updating: content/task1_default/ (stored 0%)
updating: content/task1_default/special_tokens_map.json (deflated 77%)
updating: content/task1_default/adapter_model.safetensors (deflated 8%)
updating: content/task1_default/chat_template.jinja (deflated 66%)
updating: content/task1_default/adapter_config.json (deflated 58%)
updating: content/task1_default/tokenizer.json (deflated 85%)
updating: content/task1_default/tokenizer_config.json (deflated 96%)
updating: content/task1_default/tokenizer.model (deflated 61%)
updating: content/task1_default/README.md (deflated 65%)


In [10]:
from google.colab import files
files.download("/content/task1_default.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
del model, tokenizer
torch.cuda.empty_cache()

In [22]:
from unsloth.chat_templates import get_chat_template

model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [23]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Forces Mistral v0.3 format
    )

In [24]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_1_clarity', 'persona', tokenizer, "/content/prompts.yaml")

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [25]:
train_model(model, tokenizer, dataset=subset, output_dir= "task1_persona")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.691100
2,2.434100
3,2.597900
4,2.203600
5,2.269400
6,2.047800
7,1.922200
8,1.828200
9,1.869800
10,1.891700


Saving adapter to task1_persona


In [26]:
!zip -r /content/task1_persona.zip /content/task1_persona/

  adding: content/task1_persona/ (stored 0%)
  adding: content/task1_persona/special_tokens_map.json (deflated 77%)
  adding: content/task1_persona/adapter_model.safetensors (deflated 8%)
  adding: content/task1_persona/chat_template.jinja (deflated 66%)
  adding: content/task1_persona/adapter_config.json (deflated 58%)
  adding: content/task1_persona/tokenizer.json (deflated 85%)
  adding: content/task1_persona/tokenizer_config.json (deflated 96%)
  adding: content/task1_persona/tokenizer.model (deflated 61%)
  adding: content/task1_persona/README.md (deflated 65%)


In [27]:
files.download("/content/task1_persona.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
del model, tokenizer
torch.cuda.empty_cache()

In [7]:
model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Forces Mistral v0.3 format
    )

In [9]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_1_clarity', 'chain_of_thought', tokenizer, "/content/prompts.yaml")

In [10]:
train_model(model, tokenizer, dataset=subset, output_dir= "task1_cot")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.709700
2,2.446100
3,2.606400
4,2.199200
5,2.293100
6,2.067800
7,1.953500
8,1.846400
9,1.875600
10,1.893500


Saving adapter to task1_cot


In [11]:
!zip -r /content/task1_cot.zip /content/task1_cot/

  adding: content/task1_cot/ (stored 0%)
  adding: content/task1_cot/special_tokens_map.json (deflated 77%)
  adding: content/task1_cot/adapter_model.safetensors (deflated 8%)
  adding: content/task1_cot/chat_template.jinja (deflated 66%)
  adding: content/task1_cot/adapter_config.json (deflated 58%)
  adding: content/task1_cot/tokenizer.json (deflated 85%)
  adding: content/task1_cot/tokenizer_config.json (deflated 96%)
  adding: content/task1_cot/tokenizer.model (deflated 61%)
  adding: content/task1_cot/README.md (deflated 65%)


In [13]:
from google.colab import files
files.download("/content/task1_cot.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
del model, tokenizer
torch.cuda.empty_cache()

In [28]:
model, tokenizer = get_model_and_tokenizer()

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [29]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # Forces Mistral v0.3 format
    )

In [30]:
dataset = load_dataset("ailsntua/QEvasion")
subset = transform_dataset_task(dataset, 'task_1_clarity', 'definition_aware', tokenizer, "/content/prompts.yaml")

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [31]:
train_model(model, tokenizer, dataset=subset, output_dir= "task1_definition")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/345 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 345 | Num Epochs = 1 | Total steps = 44
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Step,Training Loss
1,2.681400
2,2.436000
3,2.560000
4,2.169900
5,2.217100
6,2.011000
7,1.884900
8,1.800900
9,1.833800
10,1.849300


Saving adapter to task1_definition


In [32]:
!zip -r /content/task1_definition.zip /content/task1_definition/

  adding: content/task1_definition/ (stored 0%)
  adding: content/task1_definition/special_tokens_map.json (deflated 77%)
  adding: content/task1_definition/adapter_model.safetensors (deflated 8%)
  adding: content/task1_definition/chat_template.jinja (deflated 66%)
  adding: content/task1_definition/adapter_config.json (deflated 58%)
  adding: content/task1_definition/tokenizer.json (deflated 85%)
  adding: content/task1_definition/tokenizer_config.json (deflated 96%)
  adding: content/task1_definition/tokenizer.model (deflated 61%)
  adding: content/task1_definition/README.md (deflated 65%)


In [33]:
files.download("/content/task1_definition.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from tqdm import tqdm
from collections import Counter

def predict_label(question, answer, president, date, task_type, technique, model, tokenizer):
    """Generates a classification label for a given question/answer pair."""

    # Set the correct system prompt based on the task
    prompts = load_prompts("/content/prompts.yaml")

    sys_prompt = prompts[task_type][technique]
    messages = [
        {"role": "system", "content": sys_prompt['system']},
        {"role": "user", "content": f"President: {president}\nDate: {date}\n{question}\n{answer}"},
    ]

    # Tokenize and format with the chat template
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048,
    ).to("cuda")
    # Generate the classification
    outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],  # CRITICAL: Pass the attention mask
            max_new_tokens=50,  # Reduced since we expect short labels
            do_sample=False,
            temperature=1.0,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )


    # Slice the output tensor to isolate only the newly generated tokens
    input_length = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_length:]


    # Slice the output tensor to isolate only the newly generated tokens
    # This is much cleaner and less error-prone than string splitting!
    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()


def evaluate_model_on_dataset(test_dataset, task_type, technique, model, tokenizer):
    """Iterates through the test dataset and collects predictions."""
    results = []
    task_mapping = {'Clear Reply': ['Explicit'],
                  'Ambivalent': ['Implicit', 'Dodging', 'General', 'Deflection', 'Partial/half-answer'],
                  'Clear Non-Reply': ['Declining to answer', 'Claims ignorance', 'Clarification']}

    for row in tqdm(test_dataset):
        question = row['interview_question']
        answer = row['interview_answer']
        clarity = row['clarity_label']
        president = row['president']
        date = row['date']
        ground_truth = None
        if task_type == 'task_1_clarity':
            ground_truth = clarity
        else:
            valid_evasions = task_mapping.get(clarity, [])

            # Gather the 3 annotations
            raw_votes = [row['annotator1'], row['annotator2'], row['annotator3']]

            # Filter step: only keep votes that exist in the valid mapping
            valid_votes = [vote for vote in raw_votes if vote in valid_evasions]

            if valid_votes == []:
                # Fallback for Zero Valid Votes: Pick the most common evasion tactic for that clarity class
                fallbacks = {
                    'Clear Reply': 'Explicit',
                    'Ambivalent': 'Dodging',
                    'Clear Non-Reply': 'Declining to answer'
                }
                evasion = fallbacks.get(clarity, None)
            else:
                vote_counts = Counter(valid_votes)
                most_common = vote_counts.most_common()

                evasion = most_common[0][0]

            ground_truth = evasion

        prediction = predict_label(question, answer, president, date, task_type, technique, model, tokenizer)

        results.append({
            "question": question,
            "answer": answer,
            "ground_truth": ground_truth,
            "prediction": prediction
        })

    return results


In [15]:
from sklearn.metrics import f1_score, classification_report

def evaluate_f1_scores(results, valid_labels):
    """
    Parses LLM predictions from a list of result dictionaries and calculates F1-scores.
    """
    y_true = []
    y_pred = []

    for row in results:
        ground_truth = row['ground_truth']
        raw_prediction = row['prediction']

        # 1. Safely Extract the Label
        # We check if any of the valid labels are contained within the generated text.
        # This prevents errors if the model outputs "The label is: Deflection."
        parsed_pred = "Unknown" # Fallback if the model hallucinated
        for label in valid_labels:
            if label.lower() in raw_prediction.lower():
                parsed_pred = label
                break

        y_true.append(ground_truth)
        y_pred.append(parsed_pred)

    # 2. Calculate F1 Scores
    # zero_division=0 prevents warnings if a class was never predicted
    macro_f1 = f1_score(y_true, y_pred, average='macro', labels=valid_labels, zero_division=0)
    weighted_f1 = f1_score(y_true, y_pred, average='weighted', labels=valid_labels, zero_division=0)

    print(f"Macro F1-Score:    {macro_f1:.4f}")
    print(f"Weighted F1-Score: {weighted_f1:.4f}\n")

    # 3. Generate the Classification Report
    # This builds a text report showing precision, recall, and F1 for each class.
    print("--- Detailed Classification Report ---")
    print(classification_report(y_true, y_pred, labels=valid_labels, zero_division=0))

    return macro_f1, weighted_f1

# --- How to use it with your data ---



# Or for evasion:
# print("Evaluating Evasion Task:")
# evaluate_f1_scores(evasion_results, evasion_labels)

In [14]:
del model, tokenizer

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

In [ ]:
model.load_adapter("/content/task1_default", adapter_name="v1_default")
model.load_adapter("/content/task1_cot", adapter_name="v1_cot")
model.load_adapter("/content/task1_persona", adapter_name="v1_persona")
model.load_adapter("/content/task1_definition", adapter_name="v1_definition")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [22]:
res = {}

In [20]:
dataset = load_dataset("ailsntua/QEvasion")
test_dataset = dataset['test']

In [18]:
model.load_adapter("/content/task1_default", adapter_name="v1_default")

In [19]:
model.set_adapter("v1_default")
res['v1_default'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_1_clarity',
                                              technique='default', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:09<00:00,  1.01s/it]


In [34]:
del model, tokenizer
torch.cuda.empty_cache()

In [35]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

In [18]:
model.load_adapter("/content/task1_cot", adapter_name="v1_cot")

In [23]:
model.set_adapter("v1_cot")
res['v1_cot'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_1_clarity',
                                              technique='chain_of_thought', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:23<00:00,  1.05s/it]


In [31]:
model.load_adapter("/content/task1_persona", adapter_name="v1_persona")

In [32]:
model.set_adapter("v1_persona")
res['v1_persona'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_1_clarity',
                                              technique='persona', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:22<00:00,  1.05s/it]


In [36]:
model.load_adapter("/content/task1_definition", adapter_name="v1_definition")

In [37]:
model.set_adapter("v1_definition")
res['v1_definition'] = evaluate_model_on_dataset(test_dataset=test_dataset, task_type='task_1_clarity',
                                              technique='definition_aware', model=model, tokenizer=tokenizer)

100%|██████████| 308/308 [05:20<00:00,  1.04s/it]


In [24]:
clarity_labels = ['Clear Reply', 'Clear Non-Reply', 'Ambivalent']

In [20]:
# Define the exact labels your model was trained to output
clarity_labels = ['Clear Reply', 'Clear Non-Reply', 'Ambivalent']
# Assuming 'results' is the list returned from your inference loop:
print("Evaluating clarity Task (default):")
f1 = evaluate_f1_scores(res['v1_default'], clarity_labels)

Evaluating clarity Task (default):
Macro F1-Score:    0.1532
Weighted F1-Score: 0.1407

--- Detailed Classification Report ---
                 precision    recall  f1-score   support

    Clear Reply       0.26      0.96      0.40        79
Clear Non-Reply       0.00      0.00      0.00        23
     Ambivalent       0.55      0.03      0.06       206

       accuracy                           0.27       308
      macro avg       0.27      0.33      0.15       308
   weighted avg       0.43      0.27      0.14       308



In [25]:
print("Evaluating clarity Task (cot):")
f1 = evaluate_f1_scores(res['v1_cot'], clarity_labels)

Evaluating clarity Task (cot):
Macro F1-Score:    0.2067
Weighted F1-Score: 0.2408

--- Detailed Classification Report ---
                 precision    recall  f1-score   support

    Clear Reply       0.27      0.94      0.42        79
Clear Non-Reply       0.00      0.00      0.00        23
     Ambivalent       0.67      0.12      0.20       206

       accuracy                           0.32       308
      macro avg       0.31      0.35      0.21       308
   weighted avg       0.52      0.32      0.24       308



In [33]:
print("Evaluating clarity Task (persona):")
f1 = evaluate_f1_scores(res['v1_persona'], clarity_labels)

Evaluating clarity Task (persona):
Macro F1-Score:    0.1732
Weighted F1-Score: 0.1773

--- Detailed Classification Report ---
                 precision    recall  f1-score   support

    Clear Reply       0.26      0.96      0.41        79
Clear Non-Reply       0.00      0.00      0.00        23
     Ambivalent       0.63      0.06      0.11       206

       accuracy                           0.29       308
      macro avg       0.30      0.34      0.17       308
   weighted avg       0.49      0.29      0.18       308



In [38]:
print("Evaluating clarity Task (definition_aware):")
f1 = evaluate_f1_scores(res['v1_definition'], clarity_labels)

Evaluating clarity Task (definition_aware):
Macro F1-Score:    0.1347
Weighted F1-Score: 0.1037

--- Detailed Classification Report ---
                 precision    recall  f1-score   support

    Clear Reply       0.25      0.99      0.40        79
Clear Non-Reply       0.00      0.00      0.00        23
     Ambivalent       0.00      0.00      0.00       206

       accuracy                           0.25       308
      macro avg       0.08      0.33      0.13       308
   weighted avg       0.07      0.25      0.10       308



In [39]:
!zip -r /content/task1_default.zip /content/task1_default/

updating: content/task1_default/ (stored 0%)
updating: content/task1_default/special_tokens_map.json (deflated 77%)
updating: content/task1_default/adapter_model.safetensors (deflated 8%)
updating: content/task1_default/chat_template.jinja (deflated 66%)
updating: content/task1_default/adapter_config.json (deflated 58%)
updating: content/task1_default/tokenizer.json (deflated 85%)
updating: content/task1_default/tokenizer_config.json (deflated 96%)
updating: content/task1_default/tokenizer.model (deflated 61%)
updating: content/task1_default/README.md (deflated 65%)
